In [7]:
import pandas as pd
df= pd.read_json('datasets/subtaskA_test_monolingual.jsonl', lines=True)
df= pd.read_json('datasets/subtaskB_test.jsonl', lines=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18000 entries, 0 to 17999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    18000 non-null  object
 1   id      18000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 281.4+ KB


In [8]:
df.sample(5)

,text,id
923,Driverless vehicles are becoming more commonpl...,923
13270,"The ""face on Mars"" is at the center of controv...",13270
5281,The use of technology to monitor students' em...,5281
16889,In the arcticle it states how driverless cars ...,16889
5032,"Now that everyone has a cell phone, usage whil...",5032


In [9]:
df.head(5)

,text,id
0,Decision-making is one of life's most difficul...,0
1,"When you think of voting for president, do you...",1
2,Driverless Cars - Limitations & Potential Draw...,2
3,There are multiple benefits to attending high ...,3
4,"The Electoral College, also known as the Presi...",4


In [10]:
df['text_length'] = df['text'].apply(len)
df['text_length'].describe()

count    18000.000000
mean      2381.772833
std       1027.617891
min         76.000000
25%       1729.000000
50%       2405.000000
75%       2864.000000
max      17000.000000
Name: text_length, dtype: float64

In [5]:
from datasets import Dataset
import pandas as pd
import evaluate
import numpy as np
from transformers import EarlyStoppingCallback,AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding, AutoTokenizer, set_seed
import os
from sklearn.model_selection import train_test_split
from scipy.special import softmax
import argparse
import logging
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import wandb
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
import os
from torch.utils.data import Dataset
import torch
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score, confusion_matrix
from imblearn.under_sampling import RandomUnderSampler
from datasets import Dataset
import numpy as np
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from textstat.textstat import textstatistics
import pandas as pd
import language_tool_python
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.metrics import precision_score, recall_score, roc_auc_score, confusion_matrix
import numpy as np
from itertools import combinations
from sklearn.ensemble import RandomForestClassifier

c:\Users\Ghiki\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

# Initialize the grammar checker
tool = language_tool_python.LanguageTool('en-US')

def extract_text_features(text):
    # Tokenize the text
    words = word_tokenize(text)
    sentences = sent_tokenize(text)
    word_count = len(words)
    sentence_count = len(sentences)

    # Lexical diversity
    unique_words = set(words)
    lexical_diversity = len(unique_words) / word_count

    # Average sentence length
    avg_sentence_length = word_count / sentence_count

    # Average word length
    avg_word_length = sum(len(word) for word in words) / word_count

    # Lexical density
    tags = nltk.pos_tag(words)
    content_words = [word for word, tag in tags if tag.startswith(('N', 'V', 'J', 'R'))]
    lexical_density = len(content_words) / word_count

    # Readability scores
    flesch_reading_ease = textstatistics().flesch_reading_ease(text)
    fog_index = textstatistics().gunning_fog(text)

    # Grammatical errors
    matches = tool.check(text)
    grammatical_errors = len(matches)

    # Create a DataFrame
    return pd.Series({
        'word_count': word_count,
        'sentence_count': sentence_count,
        'lexical_diversity': lexical_diversity,
        'avg_sentence_length': avg_sentence_length,
        'avg_word_length': avg_word_length,
        'lexical_density': lexical_density,
        'flesch_reading_ease': flesch_reading_ease,
        'gunning_fog_index': fog_index,
        'grammatical_errors': grammatical_errors
    })

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ghiki\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Ghiki\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [11]:
from tqdm import tqdm
tqdm.pandas()
df= pd.read_json('datasets/subtaskA_test_monolingual.jsonl', lines=True)
test_df= pd.read_json('datasets/subtaskB_test.jsonl', lines=True)
features_df = df['text'].progress_apply(extract_text_features)
features_df_test = test_df['text'].progress_apply(extract_text_features)

100%|██████████| 18000/18000 [21:25<00:00, 14.00it/s] 


In [13]:
features_df

,word_count,sentence_count,lexical_diversity,avg_sentence_length,avg_word_length,lexical_density,flesch_reading_ease,gunning_fog_index,grammatical_errors
0,542.0,31.0,0.464945,17.483871,4.077491,0.512915,80.72,8.14,6.0
1,464.0,23.0,0.551724,20.173913,5.202586,0.575431,28.13,14.96,1.0
2,558.0,18.0,0.430108,31.000000,5.464158,0.569892,17.17,16.62,3.0
3,442.0,16.0,0.604072,27.625000,5.615385,0.581448,12.56,18.99,0.0
4,1080.0,51.0,0.323148,21.176471,4.625000,0.555556,52.19,10.52,4.0
...,...,...,...,...,...,...,...,...,...
34267,371.0,15.0,0.528302,24.733333,4.175202,0.582210,59.43,11.18,15.0
34268,383.0,15.0,0.464752,25.533333,4.885117,0.519582,49.25,12.65,0.0
34269,341.0,11.0,0.360704,31.000000,5.293255,0.539589,35.00,13.28,0.0
34270,273.0,13.0,0.564103,21.000000,3.893773,0.527473,77.77,9.32,31.0


In [14]:
features_df_test

,word_count,sentence_count,lexical_diversity,avg_sentence_length,avg_word_length,lexical_density,flesch_reading_ease,gunning_fog_index,grammatical_errors
0,219.0,10.0,0.794521,21.900000,4.273973,0.534247,64.30,9.84,4.0
1,806.0,30.0,0.356079,26.866667,4.204715,0.508685,66.98,9.72,26.0
2,155.0,5.0,0.903226,31.000000,5.232258,0.625806,33.78,18.19,3.0
3,499.0,19.0,0.258517,26.263158,4.709419,0.631263,48.33,11.34,2.0
4,369.0,16.0,0.466125,23.062500,4.457995,0.506775,58.82,11.44,3.0
...,...,...,...,...,...,...,...,...,...
17995,639.0,28.0,0.431925,22.821429,4.244131,0.528951,67.08,11.13,9.0
17996,464.0,19.0,0.463362,24.421053,5.068966,0.525862,32.53,14.68,0.0
17997,235.0,11.0,0.804255,21.363636,4.706383,0.574468,53.61,10.83,5.0
17998,530.0,22.0,0.477358,24.090909,5.658491,0.603774,15.81,15.63,0.0


In [15]:
features_df.to_csv('datasets/features_df_test_A.csv', index=False)
features_df_test.to_csv('datasets/features_df_test_B.csv', index=False)